# Tworzymy nazwane klastry

W lekcji tej użyjemy LLM-ów do pomocy przy znalezieniu odpowiednich nazw i opisów dla klastrów.

In [1]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import dotenv_values
from pycaret.clustering import predict_model, load_model

In [11]:
env = dotenv_values(".env")

openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

ładujemy nasze dane

In [3]:
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
df.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


ładujemy model wytrenowany w poprzedniej lekcji

In [4]:
kmeans_pipeline = load_model('welcome_survey_clustering_pipeline_v2')

Transformation Pipeline and Model Successfully Loaded


aplikujemy model do danych

In [5]:
df_with_clusters = predict_model(model=kmeans_pipeline, data=df)
df_with_clusters["Cluster"].value_counts()

Cluster
Cluster 0    79
Cluster 1    43
Cluster 2    33
Cluster 6    32
Cluster 5    21
Cluster 7    14
Cluster 3     6
Cluster 4     1
Name: count, dtype: int64

stworzymy teraz prompt, który prześlemy do LLM-a w celu znalezienia odpowiednich nazw i opisów dla klastrów

In [6]:
cluster_descriptions = {}
for cluster_id in df_with_clusters['Cluster'].unique():
    cluster_df = df_with_clusters[df_with_clusters['Cluster'] == cluster_id]
    summary = ""
    for column in df_with_clusters:
        if column == 'Cluster':
            continue

        value_counts = cluster_df[column].value_counts()
        value_counts_str = ', '.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()])
        summary += f"{column} - {value_counts_str}\n"

    cluster_descriptions[cluster_id] = summary

In [7]:
print(cluster_descriptions["Cluster 7"])

age - 55-64: 14, 18-24: 0, 25-34: 0, 35-44: 0, 45-54: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Wyższe: 14, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 8, Inne: 3, Koty: 2, Brak ulubionych: 1, Koty i Psy: 0
fav_place - Nad wodą: 5, W lesie: 4, W górach: 1, Inne: 0
gender - Mężczyzna: 13, Kobieta: 1



In [8]:
prompt = "Użyliśmy algorytmu klastrowania."
for cluster_id, description in cluster_descriptions.items():
    prompt += f"\n\nKlaster {cluster_id}:\n{description}"

prompt += """
Wygeneruj najlepsze nazwy dla każdego z klasterów oraz ich opisy

Użyj formatu JSON. Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
}
"""
print(prompt)

Użyliśmy algorytmu klastrowania.

Klaster Cluster 4:
age - <18: 1, 18-24: 0, 25-34: 0, 35-44: 0, 45-54: 0, 55-64: 0, >=65: 0, unknown: 0
edu_level - Podstawowe: 1, Wyższe: 0, Średnie: 0
fav_animals - Brak ulubionych: 1, Inne: 0, Koty: 0, Koty i Psy: 0, Psy: 0
fav_place - Inne: 0, Nad wodą: 0, W górach: 0, W lesie: 0
gender - Kobieta: 1, Mężczyzna: 0


Klaster Cluster 1:
age - 45-54: 13, 35-44: 11, 25-34: 10, 18-24: 8, >=65: 1, 55-64: 0, <18: 0, unknown: 0
edu_level - Średnie: 43, Podstawowe: 0, Wyższe: 0
fav_animals - Psy: 20, Koty: 12, Inne: 8, Brak ulubionych: 3, Koty i Psy: 0
fav_place - W górach: 18, Nad wodą: 12, W lesie: 7, Inne: 3
gender - Mężczyzna: 37, Kobieta: 6


Klaster Cluster 2:
age - 25-34: 20, 45-54: 13, 18-24: 0, 35-44: 0, 55-64: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Wyższe: 33, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 27, Inne: 6, Brak ulubionych: 0, Koty: 0, Koty i Psy: 0
fav_place - W lesie: 20, W górach: 7, Nad wodą: 4, Inne: 1
gender - Mężczyzna: 23, Kobi

In [13]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
)

In [14]:
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions = json.loads(result)

In [15]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [16]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "r") as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Profesjonaliści nad wodą', 'description': 'W tym klastrze znajdują się osoby w wieku 35-54 lat, z wyższym wykształceniem, które preferują psy i spędzanie czasu nad wodą. Większość z nich to mężczyźni.'}, 'Cluster 1': {'name': 'Średnio wykształceni miłośnicy gór', 'description': 'W tym klastrze znajdują się osoby w różnym wieku, głównie mężczyźni, z średnim wykształceniem, które preferują psy i spędzanie czasu w górach.'}, 'Cluster 2': {'name': 'Młodzi profesjonaliści leśni', 'description': 'W tym klastrze znajdują się młodsze osoby w wieku 25-34 lat, z wyższym wykształceniem, które preferują psy i spędzanie czasu w lesie. Większość z nich to mężczyźni.'}, 'Cluster 3': {'name': 'Miłośnicy kotów i psów nad wodą', 'description': 'W tym klastrze znajdują się osoby w wieku 35-54 lat, z wyższym wykształceniem, które lubią zarówno koty, jak i psy, oraz preferują spędzanie czasu nad wodą. Skład klastru jest równomiernie podzielony między kobiety i mężczyzn.'}, 'Cluster 